In [1]:
!pip install --quiet pytorch-crf

In [2]:
import os
import random
import numpy as np

import functools

import torch
import torch.nn as nn

from torchtext import datasets
from torchtext.data import Field
from torchtext.data import BucketIterator

from torchcrf import CRF


SEED = 241

In [3]:
def seed_everything(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)

  if torch.cuda.is_available(): 
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [4]:
TEXT = Field(lower=True,
             use_vocab=True,
             sequential=True,
             batch_first=True,
             include_lengths=True)

LABEL = Field(lower=True,
              use_vocab=True,
              sequential=True,
              unk_token = None,
              batch_first=True)

In [5]:
fields = [('text', TEXT), ('tags', LABEL)]

train_data, valid_data, test_data = datasets.UDPOS.splits(fields)

en-ud-v2.zip: 100%|██████████| 688k/688k [00:00<00:00, 34.6MB/s]

downloading en-ud-v2.zip
extracting


In [6]:
TEXT.build_vocab(train_data,
                 max_size=25000,
                 vectors='glove.6B.100d')
LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:29, 2.22MB/s]                           
100%|█████████▉| 398570/400000 [00:15<00:00, 24772.01it/s]

In [8]:
batch_size = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')


train_iterator = BucketIterator.splits((train_data,), batch_size=batch_size, device=device)[0]
valid_iterator = BucketIterator.splits((valid_data,), batch_size=batch_size, device=device)[0]
test_iterator = BucketIterator.splits((test_data,), batch_size=batch_size, device=device)[0]

In [9]:
class BiLSTM_CRF_Tagger(nn.Module):

  def __init__(self, vocab_size, emb_size, hidden_size, n_layers, dropout, num_tags, pad_idx):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=pad_idx)
    self.rnn = nn.LSTM(emb_size,
                       hidden_size, 
                       num_layers=n_layers,
                       dropout=0.3, 
                       bidirectional=True, 
                       batch_first=True)
    
    self.dropout = nn.Dropout(dropout)
    self.hidden2tag = nn.Linear(emb_size, num_tags)
    self.crf = CRF(num_tags, batch_first=True)

  def _generate_mask(self, text_lens):
    bs = text_lens.size(0)
    max_seq_len = torch.max(text_lens).item()
    mask = torch.ByteTensor(bs, max_seq_len).fill_(0)
    for i in range(bs):
      mask[i, :text_lens[i]] = 1
    return mask

  def forward(self, text, text_lens, tags=None):
    text_embed = self.embedding(text)

    text_packed = nn.utils.rnn.pack_padded_sequence(text_embed, text_lens, batch_first=True, enforce_sorted=False)
    rnn_outputs, (last_hidden, cell_state) = self.rnn(text_packed)
    text_unpacked, lens_unpacked = nn.utils.rnn.pad_packed_sequence(text_packed, batch_first=True)
    last_hidden = last_hidden.permute(1, 0, 2)

    emission = self.hidden2tag(text_unpacked)
    mask = self._generate_mask(text_lens).to(device)

    if tags is not None:
      loss = -self.crf.forward(torch.log_softmax(emission, dim=2), tags, mask, reduction='mean')
      return loss
    else:
      prediction = self.crf.decode(emission, mask)
      return prediction

In [10]:
VOCAB_SIZE = len(TEXT.vocab)
EMB_SIZE = 100
HIDDEN_SIZE = 128
N_LAYERS = 2
DROPOUT = 0.3
NUM_TAGS = len(LABEL.vocab)
PAD_IDX = LABEL.vocab.stoi['<pad>']


model = BiLSTM_CRF_Tagger(VOCAB_SIZE, EMB_SIZE, HIDDEN_SIZE, N_LAYERS, DROPOUT, NUM_TAGS, PAD_IDX)
optimizer = torch.optim.Adam(model.parameters())

In [11]:
model.embedding.weight.data.copy_(TEXT.vocab.vectors)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2634,  0.0742, -0.1081,  ..., -0.2977, -0.5655,  0.5218],
        [ 0.4244,  0.6004, -0.1528,  ...,  0.2536, -0.4969,  0.8964]])

In [12]:
model = model.to(device)

In [13]:
from tqdm import tqdm


for i in range(10):
  model.train()
  error = 0.

  for batch in tqdm(train_iterator):
    optimizer.zero_grad()

    text, lens = batch.text
    lens = lens.cpu()
    tags = batch.tags
    
    loss = model(text, lens, tags)
    loss.backward()

    optimizer.step()
    error += loss.detach().cpu().numpy()
  print('train error', error / len(train_iterator))

  error = 0.
  model.eval()
  with torch.no_grad():
    for batch in tqdm(valid_iterator):
      text, lens = batch.text
      lens = lens.cpu()
      tags = batch.tags
      
      loss = model(text, lens, tags)
      error += loss.detach().cpu().numpy()
    print('valid error', error / len(valid_iterator))


100%|██████████| 392/392 [00:27<00:00, 14.30it/s]

  8%|▊         | 5/63 [00:00<00:01, 48.51it/s]

train error 19.54022534526124



100%|██████████| 63/63 [00:01<00:00, 46.51it/s]

  1%|          | 2/392 [00:00<00:27, 14.09it/s]

valid error 7.441745409889827



100%|██████████| 392/392 [00:26<00:00, 14.81it/s]

  8%|▊         | 5/63 [00:00<00:01, 42.22it/s]

...

valid error 3.9766852855682373



100%|██████████| 392/392 [00:26<00:00, 14.95it/s]

  8%|▊         | 5/63 [00:00<00:01, 45.10it/s]

train error 2.22788618900338



100%|██████████| 63/63 [00:01<00:00, 47.58it/s]

valid error 3.963253762986925


In [14]:
def calculate_accuracy(y_true, y_pred):
  assert y_true.shape == y_pred.shape
  assert len(y_true.shape) == 1
  y_true = y_true[1:]
  y_pred = y_pred[1:]
  return (y_true == y_pred).sum() / y_true.shape[0]

y_true_test = np.array([-1, 1, 2, 3])
y_pred_test = np.array([-1, 1, 0, 3])

calculate_accuracy(y_true_test, y_pred_test)

0.6666666666666666

In [20]:
total_true_labels = []
total_pred_labels = []

for index in range(len(test_data.examples)):

  text = test_data.examples[index].text
  true_labels = test_data.examples[index].tags

  with torch.no_grad():
    tokens = text
    ids = [TEXT.vocab.stoi[token] for token in tokens]
    ids_tensor = torch.tensor([ids], device=device)
    lens = torch.tensor([len(ids)])
    prediction = model(ids_tensor, lens)
    
  print('\t'.join(tokens))
  print('\t'.join(true_labels))
  print('\t'.join([LABEL.vocab.itos[p] for p in prediction[0]]))
  print('='*20)

  total_true_labels.extend(np.array([LABEL.vocab.itos[p] for p in prediction[0]]))
  total_pred_labels.extend(np.array(true_labels))

Выходные данные были обрезаны до нескольких последних строк (5000).
pairing	up	of	harry	and	ginny	is	cool
noun	noun	adp	propn	cconj	propn	aux	adj
verb	adp	adp	propn	cconj	noun	aux	adj
it	s	awesome	-	better	than	book	5
pron	aux	adj	punct	adj	adp	noun	num
pron	aux	adj	punct	adj	sconj	verb	num
an	orphaned	,	two	-	month	old	african	elephant	named	olly	received	an	extremely	uplifting	christmas	present	this	year	:	an	airplane	ride	just	for	him	,	courtesy	of	the	international	fund	for	animal	welfare	and	their	friends	'	the	bateleurs	'	.
det	verb	punct	num	punct	noun	adj	adj	noun	verb	propn	verb	det	adv	adj	propn	noun	det	noun	punct	det	noun	noun	adv	adp	pron	punct	noun	adp	det	propn	propn	adp	propn	propn	cconj	pron	noun	punct	det	propn	punct	punct
det	noun	punct	num	punct	noun	adj	adj	noun	verb	part	verb	det	adv	propn	propn	verb	det	noun	punct	det	noun	verb	adv	adp	pron	punct	noun	adp	det	adj	noun	adp	noun	noun	cconj	pron	noun	punct	det	noun	punct	punct
animal	news	center	webmaster
propn	prop

In [22]:
calculate_accuracy(np.array(total_true_labels), np.array(total_pred_labels))

0.902414727446605

In [21]:
from sklearn.metrics import classification_report


print(classification_report(np.array(total_true_labels), np.array(total_pred_labels)))

              precision    recall  f1-score   support

         adj       0.86      0.87      0.86      1688
         adp       0.96      0.92      0.94      2119
         adv       0.85      0.89      0.87      1164
         aux       0.97      0.91      0.94      1599
       cconj       0.99      0.99      0.99       734
         det       0.99      0.98      0.98      1908
        intj       0.78      0.95      0.85        98
        noun       0.89      0.84      0.87      4380
         num       0.62      0.94      0.75       354
        part       0.96      0.90      0.93       670
        pron       0.96      0.98      0.97      2129
       propn       0.73      0.76      0.74      1981
       punct       0.99      1.00      0.99      3100
       sconj       0.74      0.80      0.77       357
         sym       0.78      0.99      0.87        73
        verb       0.89      0.89      0.89      2651
           x       0.19      0.29      0.23        92

    accuracy              